In [1]:
import os
import cv2
import shutil
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from skimage.segmentation import clear_border
from skimage.morphology import binary_closing, disk
from scipy.ndimage import binary_dilation, binary_fill_holes

images = '../images/gbif_images'
save_dir = "../data/to_edit"
image_save_dir = "../data/dataset/images"
label_save_dir = "../data/dataset/labels"

os.makedirs(save_dir, exist_ok=True)
os.makedirs(image_save_dir, exist_ok=True)
os.makedirs(label_save_dir, exist_ok=True)

KERNEL_SIZE = (10, 10) # for morphological closing
MIN_AREA = 150 # for removing objects smaller than this area
CLEAR_BORDER = True # clear border objects | IMPORTANT: set to False if you have objects touching the border

Create masks to edit and move images to dataset folder

In [4]:
for image in os.listdir(images):
    # Load the image in rgb
    orig = cv2.imread(os.path.join(images, image))

    # convert to LAB color space
    lab = cv2.cvtColor(orig, cv2.COLOR_BGR2LAB)

    # apply CLAHE to the L-channel (lightness channel) -> removes shadows (approx.)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # merge the CLAHE enhanced L-channel with the original A and B channel -> convert back to BGR
    limg = cv2.merge((cl, a, b))
    limg_bgr = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # apply heatmap with cool
    heatmap = cv2.applyColorMap(limg_bgr, cv2.COLORMAP_COOL)

    # Convert the image to grayscale
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding to segment the image
    _, thresh = cv2.threshold(heatmap_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Apply median blur to remove noise
    # thresh = cv2.medianBlur(thresh, 21)

    ## The code below is used to apply morphological closing to the image
    # Define the structuring element (kernel)
    # kernel = np.ones(KERNEL_SIZE, np.uint8)

    # # Apply morphological closing
    # closing_image = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Convert the masks to images and save
    Image.fromarray(thresh).save(os.path.join(save_dir, os.path.basename(image).replace('.jpg', '_mask.jpg')), "JPEG")
    
    # copy the original image to the image_save_dir
    shutil.copy(os.path.join(images, image), os.path.join(image_save_dir, os.path.basename(image)))

Now manually edit using photo editing tools (remove the non masked white parts and connect the parts of the plant where the tape was)

Generate the segmentation points on the edited foreground picture

In [22]:
os.makedirs('../data/review', exist_ok=True)

for idx, mask in enumerate(os.listdir(save_dir)):

    # Load the mask image in grayscale mode
    image = Image.open(os.path.join(save_dir, mask)).convert("L")
    image_width, image_height = image.size
    image_array = np.array(image)
    
    # Convert the image to a binary mask
    image_array[image_array > 100] = 255
    image_array[image_array <= 100] = 0
    image_array_uint8 = np.uint8(image_array)

    # Clear the border of the mask
    if CLEAR_BORDER:
        image_array_uint8 = clear_border(image_array_uint8)

    # Apply binary dilation to the mask
    selem = disk(1)
    image_array_bool = binary_dilation(image_array_uint8, selem)

    # Fill the holes in the mask
    image_array_bool = binary_fill_holes(image_array_bool)

    # Apply morphological closing to the mask to smooth the edges
    selem = disk(5)
    image_array_bool = binary_closing(image_array_bool, selem)    

    # Convert the mask back to uint8
    image_array_uint8 = np.uint8(image_array_bool)
    
    # Find the contours in the binary mask and only keep the largest ones
    contours, _ = cv2.findContours(image_array_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    contours = [contour for contour in contours if cv2.contourArea(contour) > MIN_AREA]
    
    # because the contours draw so many points, we approximate them
    approx_contours = [cv2.approxPolyDP(contour, 0.0002 * cv2.arcLength(contour, True), True) for contour in contours]

    # for testing purposes
    image_bgr = cv2.imread(os.path.join(image_save_dir, mask.replace('_mask.jpg', '.jpg')))
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_RGB2BGR)
    cv2.drawContours(image_rgb, approx_contours, -1, (255, 0, 0), 4)
    contour_image = Image.fromarray(image_rgb)
    contour_image.save(f'../data/review/{mask}')
    
    with open(os.path.join(label_save_dir, mask.replace('_mask.jpg', '.txt')), "w") as f:
        for contour in approx_contours:
            normalized_contour_points = contour[:, 0, :] / [image_width, image_height]
            normalized_contour_xyxyxy_format = normalized_contour_points.flatten().tolist()
            f.write('0 ')
            f.write(" ".join(map(str, normalized_contour_xyxyxy_format)))
            f.write('\n')